In [3]:
import cv2
import mediapipe as mp
import time
import math

In [4]:
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands

def where_finger(input_tuple):
   if (input_tuple[0] > 83 or input_tuple[0] <= 30) and (input_tuple[1] > 28):
      return 'bent fully'
   if (input_tuple[0] > 30 and input_tuple[0] < 70) and (input_tuple[1] > 8):
      return 'bent'
   if (input_tuple[0] <= 83 and input_tuple[0] >= 70):
      return 'upright'

def extract_elements(input_list):
    list1 = []
    list2 = []
    list3 = []
    list4 = []

    # Iterate over batches of 20 elements
    for chunk in range(0, len(input_list), 3):
        # Iterate over each element in the current batch
        for i in range(chunk, min(chunk + 3, len(input_list))):
            item = input_list[i]
            if i % 3 == 0 or i % 3 == 1 or i % 3 == 2:
                list1.append([item[1], item[2]])


    return list1

def iterate_in_batches(list1):
  baseoffinger_pointer_y = []
  baseoffinger_pointer_z = []
  middleoffinger_pointer_y = []
  middleoffinger_pointer_z = []
  tipoffinger_pointer_y = []
  tipoffinger_pointer_z = []
  
  for i in range(0, len(list1), 3):
    for j in range(i, min(i + 3, len(list1))):
      item = list1[j]
      if j % 3 == 0:
        baseoffinger_pointer_y.append(item[0])
        baseoffinger_pointer_z.append(item[1])
      if j % 3 == 1:
        middleoffinger_pointer_y.append(item[0])
        middleoffinger_pointer_z.append(item[1])
      if j % 3 == 2:
        tipoffinger_pointer_y.append(item[0])
        tipoffinger_pointer_z.append(item[1])

  y_delta_base_mid_pointer = abs(baseoffinger_pointer_y[::-1][0]-middleoffinger_pointer_y[::-1][0])
  z_delta_base_mid_pointer = abs(baseoffinger_pointer_z[::-1][0]-middleoffinger_pointer_z[::-1][0])


  if y_delta_base_mid_pointer == 0:
    base_mid_pointer_finger_angle = 90
  elif z_delta_base_mid_pointer == 0:
    base_mid_pointer_finger_angle = 0
  else:
    base_mid_pointer_finger_angle = math.degrees(math.atan(y_delta_base_mid_pointer/z_delta_base_mid_pointer))
  y_delta_mid_tip_pointer = abs(middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0])
  z_delta_mid_tip_pointer = abs(middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0])

  flagged = False
  flagged2 = False

  if middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0] > 0:
    print(middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0])
    flagged = True
  if middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0] > 0:
    print(middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0])
    flagged2 = True


  if base_mid_pointer_finger_angle < 90 and y_delta_mid_tip_pointer != 0 and z_delta_mid_tip_pointer != 0 and not flagged and not flagged2:
    mid_tip_pointer_finger_angle = abs(base_mid_pointer_finger_angle - math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer)))
  if base_mid_pointer_finger_angle < 90 and y_delta_mid_tip_pointer != 0 and z_delta_mid_tip_pointer != 0 and flagged and flagged2:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + 90 + math.degrees(math.atan(z_delta_mid_tip_pointer/y_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle < 90 and y_delta_mid_tip_pointer != 0 and z_delta_mid_tip_pointer != 0 and not flagged and flagged2:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle >= 90 and y_delta_mid_tip_pointer != 0 and z_delta_mid_tip_pointer != 0 and not flagged:
    mid_tip_pointer_finger_angle = math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle >= 90 and y_delta_mid_tip_pointer != 0 and z_delta_mid_tip_pointer != 0 and flagged:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + 90 + math.degrees(math.atan(z_delta_mid_tip_pointer/y_delta_mid_tip_pointer))
  if y_delta_mid_tip_pointer == 0:
    mid_tip_pointer_finger_angle = 0
  if z_delta_mid_tip_pointer == 0:
    mid_tip_pointer_finger_angle = 90


  return(base_mid_pointer_finger_angle, mid_tip_pointer_finger_angle)

hands = mp_hands.Hands()
landmarks_list = []
landmarks_list_2 = []
landmarks_list_3 = []
landmarks_list_4 = []
while True:
    sucess, img = cap.read()
    #img = cv2.flip(img, 0)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks_list.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z*1000)))
            landmarks_list.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].z*1000)))
            landmarks_list.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].z*1000)))
            landmarks_list_2.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].z*1000)))
            landmarks_list_2.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].z*1000)))
            landmarks_list_2.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].z*1000)))
            landmarks_list_3.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].z*1000)))
            landmarks_list_3.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].z*1000)))
            landmarks_list_3.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].z*1000)))
            landmarks_list_4.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].z*1000)))
            landmarks_list_4.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].z*1000)))
            landmarks_list_4.append((int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].x*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y*1000),int(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].z*1000)))
    cv2.imshow("hand tracking", img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    print(iterate_in_batches(extract_elements(landmarks_list)), iterate_in_batches(extract_elements(landmarks_list_2)), iterate_in_batches(extract_elements(landmarks_list_3)), iterate_in_batches(extract_elements(landmarks_list_4)))


# output (index finger bend, curl),(middle finger bend, curl), (ring finger bend, curl), (pinky finger bend, curl)

cap.release()
cv2.destroyAllWindows()



(70.70995378081128, 4.613653081738704) (72.39536666333599, 4.212135582912907) (75.54757174890702, 1.2795765340143248) (77.0053832080835, 8.806792694435316)
(71.38983527217266, 3.901860828144592) (73.97970769792879, 3.7250258790730726) (76.82744657667311, 0.47786571143009837) (78.51800864525191, 9.213459379315182)
(69.50740523012972, 4.178742343609727) (70.70995378081128, 3.5388725557356935) (74.84593194968741, 1.545176183681022) (75.34324888423959, 8.972626614896399)
(69.31591973205319, 4.179718886191793) (70.01689347810003, 4.727987818842195) (73.5778125016847, 0.5053255658317397) (75.34324888423959, 10.426842890430507)
(68.29946326068254, 3.904547815387076) (69.63839672461386, 4.416207374463298) (73.73979529168804, 1.6677150536952752) (74.52136183458164, 9.144926620745252)
(68.92250588301327, 2.4279553629353217) (69.96898731910227, 3.886673900614241) (73.81079374297306, 1.9888125413915532) (75.1243179983612, 9.303425145050383)
(68.60710964801855, 4.038865715720121) (69.63839672461386

In [5]:
#dw about this this is all debuig
print(landmarks_list)

[(698, 439, -205), (684, 279, -137), (658, 332, -69), (694, 431, -217), (682, 274, -142), (663, 336, -73), (695, 435, -228), (684, 276, -154), (662, 334, -82), (697, 439, -227), (681, 280, -151), (660, 334, -79), (695, 447, -234), (683, 284, -157), (663, 336, -83), (694, 443, -228), (682, 283, -151), (662, 337, -78), (702, 429, -214), (684, 277, -137), (661, 340, -67), (698, 436, -231), (685, 277, -152), (664, 333, -78), (698, 432, -232), (684, 276, -153), (663, 333, -79), (696, 433, -213), (683, 278, -137), (662, 335, -68), (699, 432, -225), (684, 275, -147), (663, 335, -74), (698, 431, -220), (683, 275, -143), (662, 336, -71), (700, 431, -231), (684, 278, -152), (662, 336, -77), (701, 438, -229), (682, 281, -151), (661, 338, -76), (699, 437, -250), (682, 281, -169), (662, 337, -91), (699, 450, -208), (685, 290, -134), (663, 344, -65), (694, 452, -210), (685, 289, -137), (663, 343, -69), (690, 474, -222), (682, 306, -148), (664, 355, -77), (688, 469, -218), (680, 303, -147), (663, 353

In [8]:
#this is also debug
import math
def extract_elements(input_list):
    list1 = []
    list2 = []
    list3 = []
    list4 = []

    # Iterate over batches of 20 elements
    for chunk in range(0, len(input_list), 21):
        # Iterate over each element in the current batch
        for i in range(chunk, min(chunk + 21, len(input_list))):
            item = input_list[i]
            if i % 21 == 5 or i % 21 == 6 or i % 21 == 8:
                list1.append([item[1], item[2]])
            elif i % 21 == 9 or i % 21 == 10 or i % 21 == 12:
                list2.append([item[1], item[2]])
            elif i % 21 == 13 or i % 21 == 14 or i % 21 == 16:
                list3.append([item[1], item[2]])
            elif i % 21 == 17 or i % 21 == 18 or i % 21 == 20:
                list4.append([item[1], item[2]])


    return list1, list2, list3, list4

list_2 = []
list_2.append((1,2))
list_2.append((2,3))
print(list_2)
list1 = [(541,-132), (393, -78), (506, -30)]
# for this use case we have gotta reflect the middle point over the line joining the first and third points
#result = extract_elements(landmarks_list)




def iterate_in_batches(list1, list2, list3, list4):
  baseoffinger_pointer_y = []
  baseoffinger_pointer_z = []
  middleoffinger_pointer_y = []
  middleoffinger_pointer_z = []
  tipoffinger_pointer_y = []
  tipoffinger_pointer_z = []
  
  for i in range(0, len(list1), 3):
    for j in range(i, min(i + 3, len(list1))):
      item = list1[j]
      if j % 3 == 0:
        baseoffinger_pointer_y.append(item[0])
        baseoffinger_pointer_z.append(item[1])
      if j % 3 == 1:
        middleoffinger_pointer_y.append(item[0])
        middleoffinger_pointer_z.append(item[1])
      if j % 3 == 2:
        tipoffinger_pointer_y.append(item[0])
        tipoffinger_pointer_z.append(item[1])

  y_delta_base_mid_pointer = abs(baseoffinger_pointer_y[::-1][0]-middleoffinger_pointer_y[::-1][0])
  z_delta_base_mid_pointer = abs(baseoffinger_pointer_z[::-1][0]-middleoffinger_pointer_z[::-1][0])



  base_mid_pointer_finger_angle = math.degrees(math.atan(y_delta_base_mid_pointer/z_delta_base_mid_pointer))
  y_delta_mid_tip_pointer = abs(middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0])
  z_delta_mid_tip_pointer = abs(middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0])

  flagged = False
  flagged2 = False

  if middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0] > 0:
    print(middleoffinger_pointer_z[::-1][0]-tipoffinger_pointer_z[::-1][0])
    flagged = True
  if middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0] > 0:
    print(middleoffinger_pointer_y[::-1][0]-tipoffinger_pointer_y[::-1][0])
    flagged2 = True


  print(y_delta_mid_tip_pointer)
  print(z_delta_mid_tip_pointer)

  if base_mid_pointer_finger_angle < 90 and not flagged and not flagged2:
    mid_tip_pointer_finger_angle = abs(base_mid_pointer_finger_angle - math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer)))
  if base_mid_pointer_finger_angle < 90 and flagged and flagged2:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + 90 + math.degrees(math.atan(z_delta_mid_tip_pointer/y_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle < 90 and not flagged and flagged2:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle >= 90 and not flagged:
    mid_tip_pointer_finger_angle = math.degrees(math.atan(y_delta_mid_tip_pointer/z_delta_mid_tip_pointer))
  if base_mid_pointer_finger_angle >= 90 and flagged:
    mid_tip_pointer_finger_angle = base_mid_pointer_finger_angle + 90 + math.degrees(math.atan(z_delta_mid_tip_pointer/y_delta_mid_tip_pointer))


  return(base_mid_pointer_finger_angle, mid_tip_pointer_finger_angle)




print(iterate_in_batches(list1, [], [], []))

[(1, 2), (2, 3)]
113
48
(69.954750574579, 2.9695049933689717)
